In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score, cross_validate, cross_val_predict, GridSearchCV, train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from tqdm import tqdm
from config import *
from utils import *
import warnings
warnings.filterwarnings("ignore")

In [2]:
''' Read data & split to train / test '''
data_A = pd.read_csv('data/data_A.csv', index_col=0)
data_B = pd.read_csv('data/data_B.csv', index_col=0)
data = pd.concat([data_A, data_B], axis=0)

X_train = data_A[INPUT_VARS]
y_train = data_A[RESPONSE_VARS]
X_test = data_B[INPUT_VARS]
y_test = data_B[RESPONSE_VARS]

# # Split train to train / validation
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=32)

# print(f'X_train shape: {X_train.shape} \nX_val shape: {X_val.shape} \nX_test shape: {X_test.shape}')

### OMLT

In [3]:
from omlt import OmltBlock, OffsetScaling
from omlt.io.keras import load_keras_sequential
from omlt.neuralnet import ReluBigMFormulation, FullSpaceSmoothNNFormulation
import pyomo.environ as pyo
import pandas as pd
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

In [4]:
dfin = X_train
dfout = y_train

inputs = INPUT_VARS
outputs = RESPONSE_VARS

x_offset, x_factor = dfin.mean().to_dict(), dfin.std().to_dict()
y_offset, y_factor = dfout.mean().to_dict(), dfout.std().to_dict()

dfin = (dfin - dfin.mean()).divide(dfin.std())
dfout = (dfout - dfout.mean()).divide(dfout.std())

x = dfin.values
y = dfout.values

# capture the minimum and maximum values of the scaled inputs so we don't use the model outside the valid range
scaled_lb = dfin.min()[inputs].values
scaled_ub = dfin.max()[inputs].values

In [5]:
# create our Keras Sequential model
nn = Sequential(name='ANN')
nn.add(Dense(units=516, input_dim=len(inputs), activation='relu'))
nn.add(Dense(1))
nn.compile(optimizer=Adam(), loss='mean_absolute_error', metrics=['mean_absolute_error'])

history = nn.fit(x, y, epochs=100, verbose=0)

In [7]:
# # Get X_val transformed to ANN input
# x_val = np.array((X_val - x_offset).divide(x_factor))

# y_pred = nn.predict(x_val)

# # Get y_val transformed to ANN output
# y_pred = y_pred * y_factor['Limonene'] + y_offset['Limonene'] 

# # Create df with y_pred and y_val columns
# df = pd.DataFrame({'y_pred': y_pred.flatten(), 'y_val': y_val.values.flatten()})
# df

In [8]:
# first, create the Pyomo model
m = pyo.ConcreteModel()
# create the OmltBlock to hold the neural network model
m.reformer = OmltBlock()
# load the Keras model
nn_reformer = nn

# Note: The neural network is in the scaled space. We want access to the
# variables in the unscaled space. Therefore, we need to tell OMLT about the
# scaling factors
scaler = OffsetScaling(
        offset_inputs={i: x_offset[inputs[i]] for i in range(len(inputs))},
        factor_inputs={i: x_factor[inputs[i]] for i in range(len(inputs))},
        offset_outputs={i: y_offset[outputs[i]] for i in range(len(outputs))},
        factor_outputs={i: y_factor[outputs[i]] for i in range(len(outputs))}
    )

scaled_input_bounds = {i: (scaled_lb[i], scaled_ub[i]) for i in range(len(inputs))}

# create a network definition from the Keras model
net = load_keras_sequential(nn_reformer, scaling_object=scaler, scaled_input_bounds=scaled_input_bounds)

# create the variables and constraints for the neural network in Pyomo
m.reformer.build_formulation(ReluBigMFormulation(net))

# now add the objective and the constraints
limonene_idx = outputs.index('Limonene')
m.obj = pyo.Objective(expr=m.reformer.outputs[limonene_idx], sense=pyo.maximize)

# now solve the optimization problem (this may take some time)
solver = pyo.SolverFactory('cplex')
status = solver.solve(m, tee=False)

for i in range(len(inputs)):
    print(f'{inputs[i]}:', pyo.value(m.reformer.inputs[i]))

print('Limonene: ', pyo.value(m.reformer.outputs[limonene_idx]))

ATOB_ECOLI: 0.021899999999999975
ERG8_YEAST: 0.18989999999999896
IDI_ECOLI: 3.1812999999999985
KIME_YEAST: 0.0983000000000008
MVD1_YEAST: 0.42150000000000065
Q40322_MENSP: 11.203699999999998
Q8LKJ3_ABIGR: 4.526700000000001
Q9FD86_STAAU: 0.027999999999999803
Q9FD87_STAAU: 0.042899999999999716
Limonene:  173.93197672382112
